### Generare i JSON per le mappe: tipo 1, conto dei residenti

Obiettivo di questo notebook è di preparare i dati per  poi generare i JSON pronti per alimentare le mappe del progetto. Cominciamo con le mappe del tipo 1, che contano semplicemente i residenti. La spiegazione è [sul wiki](https://github.com/spaghetti-open-data/-milano-melting-pot/wiki/2.-Metodologia-e-workflow).

Anzitutto verifichiamo i dati sui residenti. Il file JSON sembra corrotto. Controlliamo il CSV:

In [2]:
import csv
dirPath = '/Users/albertocottica/Documents/GitHub/-milano-melting-pot/dati/'
data = [] # we know from JSON that 
with open(dirPath + 'ds27_pop_sto_quartiere.csv') as datafile:
    reader = csv.DictReader(datafile, delimiter=';')
    for row in reader:
        drow = dict(row)
        drow['Anno'] = drow.pop('\ufeffAnno') # questo è per togliere spazzatura che confonde il nome del campo
        data.append(drow)
print (len(data))

2136906


In [3]:
print(data[0])

{'IdNil': '1', 'Nil': 'Duomo', 'Eta': '0', 'Genere': 'Femmine', 'Cittadinanza': 'Cinese, Rep. Popolare', 'Residenti': '1', 'Anno': '1999'}


In [4]:
quartieri = [] # quanti e quali quartieri abbiamo?
missingRecords = 0 
for item in data:
    if 'Nil' in item:        
        if item['Nil'] not  in quartieri:
            quartieri.append(item['Nil'])
    else:
        missingRecords += 1

print (quartieri)
print ('Gli ID di quartiere a Milano sono ' + str(len(quartieri)))
print ('Bad records: ' + str(missingRecords) + ' su ' + str(len(data)))

['Duomo', 'Brera', 'Giardini Porta Venezia', 'Guastalla', 'Vigentina', 'Ticinese', 'Magenta - S. Vittore', 'Parco Sempione', 'Garibaldi Repubblica', 'Centrale', 'Isola', 'Maciachini - Maggiolina', 'Greco', 'Niguarda - Cà Granda', 'Bicocca', 'Vle Monza', 'Adriano', 'Parco Lambro - Cimiano', 'Padova', 'Loreto', 'Buenos Aires - Venezia', 'Città Studi', 'Lambrate', 'Parco Forlanini - Ortica', 'Corsica', 'XXII Marzo', 'Porta Romana', 'Umbria - Molise', 'Ortomercato', 'Mecenate', 'Parco Monlué - Ponte Lambro', 'Triulzo Superiore', 'Rogoredo', 'Chiaravalle', 'Lodi - Corvetto', 'Scalo Romana', 'Ex OM - Morivione', 'Ripamonti', 'Quintosole', 'Ronchetto delle Rane', 'Gratosoglio - Ticinello', 'Stadera', 'Tibaldi', 'Navigli', 'S. Cristoforo', 'Barona', 'Cantalupa', 'Ronchetto S/N', 'Giambellino', 'Tortona', 'Washington', 'Bande Nere', 'Lorenteggio', 'Muggiano', 'Baggio', 'Forze Armate', 'Selinunte', 'De Angeli - Monte Rosa', 'Tre Torri', 'S. Siro', 'Quarto Cagnino', 'Quinto Romano', 'Figino', 'Tr

In [5]:
anni = [] # quanti e quali anni abbiamo?
missingRecords = 0 
for item in data:
    if 'Anno' in item:        
        if item['Anno'] not  in anni:
            anni.append(item['Anno'])
    else:
        missingRecords += 1

print (anni)
print ('Anni nel dataset: ' + str(len(anni)))
print ('Bad records: ' + str(missingRecords) + ' su ' + str(len(data)))

['1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']
Anni nel dataset: 20
Bad records: 0 su 2136906


Controlliamo che la popolazione corrisponda

In [6]:
pop = 0
nonItaliani = 0
for item in data:
    if item['Anno'] == '2018':
        pop += int(item['Residenti'])
        if item['Cittadinanza'] != 'Italia':
            nonItaliani += int(item['Residenti'])
print(pop)
print (nonItaliani)

1395274
275818


In [7]:
nazioni = [] # quante e quali nazionalità abbiamo?
missingRecords = 0 
for item in data:
    if 'Cittadinanza' in item:        
        if item['Cittadinanza'] not  in nazioni:
            nazioni.append(item['Cittadinanza'])
    else:
        missingRecords += 1

print (nazioni)
print ('Le nazionalità presenti a Milano sono ' + str(len(nazioni)))
print ('Bad records: ' + str(missingRecords) + ' su ' + str(len(data)))

['Cinese, Rep. Popolare', 'Egitto', 'Etiopia', 'Filippine', 'Italia', 'Sri Lanka (ex Ceylon)', 'Togo', 'Perù', 'Francia', 'Libano', 'Mauritius', 'Regno Unito', 'India', 'Spagna', "Stati Uniti D'America", 'Israele', 'Turchia', 'Grecia', 'Marocco', 'Portogallo', 'Germania', 'Giappone', 'El Salvador', 'Iran', 'Bulgaria', 'Ecuador', 'Croazia', 'Bosnia-Erzegovina', 'ex Jugoslavia', 'Albania', 'Siria', 'Argentina', 'Cuba', 'Georgia', 'Dominicana, Rep.', 'Svizzera', 'Colombia', 'Eritrea', 'Romania', 'Russa, Federazione', 'Somalia', 'Ucraina', 'Venezuela', 'Belgio', 'Danimarca', 'Irlanda', 'Ungheria', 'Brasile', 'Svezia', 'Australia', 'Austria', 'Estonia', 'Dominica', 'Taiwan (ex Formosa)', 'Tunisia', 'Capo Verde', 'Ghana', 'Kenya', 'Lituania', 'Nigeria', 'Norvegia', 'Paesi Bassi', 'Ruanda', 'Thailandia', 'Giordania', 'Finlandia', 'Cile', 'Indonesia', 'Canada', 'Haiti', 'Nepal', 'Pakistan', 'Messico', 'Polonia', 'Corea del Sud, Rep.', 'Bolivia', 'Uruguay', 'Costa Rica', 'Senegal', 'Lussemburgo

A questo punto possiamo pre-aggregare i dati. 

Per la mappa 1 (vedere [il wiki](https://github.com/spaghetti-open-data/-milano-melting-pot/wiki/2.-Metodologia-e-workflow)), vogliamo sapere quante persone abbiamo di ciascuna nazionalità, in ciascun quartiere e in ciascun anno. Qundi dobbiamo sommare la variabile `'Residenti'` per ciascuna tripla di valori sulle variabili `'Anno'`, `'Nil'` e `'Cittadinanza'`. Inoltre, non dobbiamo perderci il genere.

Mi faccio un dizionario di liste di dizionari; questo serve a rendere il codice più efficiente. Questo mi servirà per generare i files JSON delle prime tre mappe. 

In [8]:
nestedData = {}
for quartiere in quartieri:
    nestedData[quartiere] = []
    for item in data:
        if item['Nil'] == quartiere:
            nestedData[quartiere].append(item)

In [9]:
test = nestedData['Duomo']

print (len(test))

31858


In [ ]:
import datetime
start_script = datetime.datetime.now()
summedData = []
for key in nestedData:
    y = nestedData[key]
    print('++++++ '+ str(key))
    for anno in anni:
        # print (anno)
        for nazione in nazioni: 
            triple = {'Anno': anno, 'Nil': key, 'Cittadinanza': nazione, 'Maschi': 0, 'Femmine': 0}
            for item in y:
                now = item ['Anno']
                people = item  ['Cittadinanza']
                if anno == now and people == nazione:
                    if item['Genere'] == 'Femmine':
                        triple['Femmine'] += int(item ['Residenti'])
                    else: 
                        triple['Maschi'] += int(item['Residenti'])
            summedData.append(triple)

print (summedData[0])
print (len(summedData))

end_script = datetime.datetime.now()
print ('Completato in ' + str(end_script-start_script))

Testiamo per vedere se funziona bene:

In [ ]:
for i in range(10):
    print(summedData[i])
print (len(summedData))

Sembra di sì. La variabile `Eta` è sparita; ora i residenti di tutte le età sono aggregati in un unico record per ciascuna nazionalità, quartiere e anno.

Ora voglio aggregare le nazionalità per regioni geografiche. In prima approssimazione, manteniamo la classificazione del progetto [Brussels Melting Pot](http://brussels-diversity.jetpack.ai/), e usiamo i nomi inglesi delle categorie per mantenere la comparabilità con esso. Questa prevede due livelli di aggregazione: regioni e continenti. Per prima cosa costruisco liste dei paesi che appartengono a ciascuna regione. Le regioni sono:

* EU 15 ([definizione](https://stats.oecd.org/glossary/detail.asp?ID=6805)), esclusa l'Italia
* EU N13, cioè i 28 paesi dell'UE meno i 15 di EU 15 
* Non-EU Europe (questo non c'è nel progetto Bruxelles)
* North Africa 
* Sub-Saharian Africa
* Eastern Asia ([definizione](https://en.wikipedia.org/wiki/East_Asia))
* Southern Asia
* Southeast Asia (non c'è nel progetto di Bruxelles, se i numeri sono piccoli lo elimino dopo)
* Western Asia
* North America
* South America
* Oceania
* Other (residuo)

I continenti sono:

* Europe
* Asia
* Africa
* North America
* South America
* Oceania

(il residuo è omesso)

Scrivo questa aggregazione come una funzione. 

In [ ]:
def assign_country(country):
    '''
    returns a tuple with the region and continent of country
    '''
    # elenco nazionalità, attribuite alla regione
    
    eu15 = ['Francia', 'Regno Unito', 'Spagna', 'Grecia', 'Portogallo', 'Germania', 'Belgio', 'Danimarca', 'Irlanda', 
        'Svezia', 'Austria', 'Paesi Bassi', 'Finlandia', 'Lussemburgo', 'Andorra', 'Monaco', 'Liechtenstein']
    eun13 = ['Bulgaria', 'Croazia', 'Romania', 'Ungheria', 'Estonia', 'Lituania', 'Polonia', 'Slovenia', 'Ceca, Rep.', 
         'Slovacchia', 'Lettonia', 'Malta', 'Cipro']
    otheu = ['Bosnia-Erzegovina', 'ex Jugoslavia', 'Albania', 'Georgia', 'Svizzera', 'Russa, Federazione', 'Ucraina', 
         'Norvegia', 'Moldova', 'Armenia', 'Islanda', 'Bielorussia', 'Azerbaigian', 'Kosovo', 'Serbia', 'Montenegro', 
         'Macedonia, ex Rep. Jugoslava']
    noraf = ['Egitto', 'Marocco', 'Tunisia', 'Libia', 'Algeria']
    ssaf = ['Etiopia', 'Togo', 'Mauritius', 'Eritrea', 'Somalia', 'Capo Verde', 'Ghana', 'Kenya', 'Nigeria', 'Ruanda', 
        'Senegal', 'Tanzania', 'Seychelles', 'Sudan', 'Sud Africa', "Costa D'Avorio", 'Camerun', 
        'Benin (ex Dahomey)', 'Burkina Faso (ex Alto Volta)', 'Zambia', 'Angola', 'Guinea', 'Gambia', 'Congo', 
        'Sierra Leone', 'Uganda', 'Burundi', 'Madagascar', 'Congo Rep. Dem (ex Zaire)', 'Gabon', 'Liberia', 
        'Mozambico', 'Mauritania', 'Mali', 'Zimbabwe (ex Rhodesia)', 'Gibuti', 'Niger', 'Ciad', 'Guinea Bissau', 
        'Centrafricana, Rep.', 'Sao Tomè e Principe', 'Malawi', 'Swaziland', 'Lesotho', 'Guinea Equatoriale', 
         'Sud Sudan, Repubblica del']
    eastas = ['Cinese, Rep. Popolare', 'Giappone', 'Taiwan (ex Formosa)', 'Corea del Sud, Rep.', 
          'Corea del Nord, Rep. Pop. Dem', 'Mongolia']
    southas = ['India', 'Bangladesh', 'Afghanistan', 'Bhutan', 'Maldive', 'Pakistan', 'Sri Lanka (ex Ceylon)', 'Nepal']
    seas = ['Indonesia', 'Malaysia', 'Vietnam', 'Singapore', 'Myanmar (ex Birmania)', 'Laos', 'Cambogia', 
        'Timor Orientale', 'Thailandia', 'Filippine']
    westas = ['Libano', 'Israele', 'Turchia', 'Iran', 'Siria', 'Iraq', 'Kazakistan', 'Uzbekistan', 'Yemen', 
          'Territori Autonomia Palestinese', 'Kuwait', 'Turkmenistan', 'Tagikistan', 'Kirghizistan', 'Oman']
    noram = ["Stati Uniti D'America", 'Canada', 'Giordania', 'Arabia Saudita']
    soam = ['Messico', 'Perù', 'El Salvador', 'Ecuador', 'Argentina', 'Cuba', 'Dominicana, Rep.', 'Colombia', 
        'Venezuela', 'Brasile', 'Dominica', 'Cile', 'Haiti', 'Bolivia', 'Uruguay', 'Costa Rica', 'Giamaica', 
       'Guatemala', 'Panama', 'Trinidad e Tobago', 'Nicaragua', 'Antigua e Barbuda', 'Suriname', 'Honduras', 
       'Bahamas', 'Barbados', 'Belize', 'Saint Kitts e Nevis', 'Paraguay']
    ocean = ['Australia', 'Nuova Zelanda']
    other = ['Altre nazioni o n.d.', 'Apolide']


    if country in eu15:
        location = ('eu15', 'europa')
    elif country in eun13:
        location = ('eun13', 'europa')
    elif country in otheu:
        location = ('otheu', 'europa')
    elif country in noraf:
        location = ('noraf', 'africa')
    elif country in ssaf:
        location = ('ssaf', 'africa')
    elif country in eastas:
        location = ('eastas', 'asia')
    elif country in southas:
        location = ('southas', 'asia')
    elif country in seas:
        location = ('seas', 'asia')
    elif country in westas:
        location = ('westas', 'asia')
    elif country in noram:
        location = ('noram', 'north america')
    elif country in soam:
        location = ('soam', 'south america')
    elif country in ocean:
        location = ('ocean', 'oceania')
    elif country in other:
        location = ('other','')
    else: 
        location = ('ita', 'italia')
    return location


Rimangono fuori dalle 176 nazionalità 
solo Italia e San Marino, che in effetti contano come indigeni a Milano. 

Aggiungo regioni e continenti al dataset, usando la mia funzione. 

In [ ]:
for item in summedData:
    nazione = item['Cittadinanza']
    item['regione'] = assign_country(nazione)[0]
    item['continente'] = assign_country(nazione)[1]

print (summedData[:4]) ## just for testing  
print (len(summedData))

Produco un file per la mappa 1, con la struttura descritta [qui](https://github.com/spaghetti-open-data/-milano-melting-pot/issues/3#issue-451241445).

In [ ]:
start_script = datetime.datetime.now()
map1Data = []
regio = ['eastas', 'noraf', 'ssaf', 'seas', 'ita', 'southas', 'soam', 'eu15', 'westas', 'noram', 'eun13', 
         'otheu', 'ocean', 'other']
for anno in anni:
    print (anno)
    for quartiere in quartieri:
        # print (quartiere)
        for regione in regio: 
            newrecord ={'Anno': anno, 'Nil': quartiere}
            for item in summedData: 
                if item['Anno'] == anno and item['Nil'] == quartiere and item['regione'] == regione:
                    newkey = item['regione'] + ' M'
                    if newkey not in newrecord:
                        newrecord[newkey] = item['Maschi']
                        newrecord[item['regione'] + ' F'] = item['Femmine']
                        newrecord[item['regione'] + ' Tot'] = item['Maschi'] + item['Femmine']
                    else:
                        newrecord[item['regione'] + ' M'] += item['Maschi']
                        newrecord[item['regione'] + ' F'] += item['Femmine']
                        newrecord[item['regione'] + ' Tot'] += (item['Maschi'] + item['Femmine'])      
            map1Data.append(newrecord)

print(map1Data[:4])

end_script = datetime.datetime.now()
print ('Completato in ' + str(end_script-start_script))

Infine, salviamo in json.

In [ ]:
import json
dirPath2 = '/Users/albertocottica/Documents/GitHub/-milano-melting-pot/dati/'
with open (dirPath2 + 'map1Data.json', 'w') as outfile:
    json.dump(compactData, outfile)
print('Done')

Per facilitare il lavoro agli sviluppatori web, mi scrivo anche un dataset "compatto", con un record per regione, quartiere e anno. In questo modo le 176 nazionalità vengono compattate in 15 regioni. Mantengo l'informazione su Maschi e Femmine, che può servire, ma aggiungo per ciascuna regione un campo `Tot`, la somma di maschi + femmine. **Devo controllare con Tommaso, Cesare e Marco che questo formato gli funzioni. Se non gli funziona, cancello la cella**.

In [ ]:
compactData2 = []
for anno in anni:
    print (anno)
    for quartiere in quartieri:
        # print (quartiere)
        newrecord ={'Anno': anno, 'Nil': quartiere}
        for item in summedData: 
            if item['Anno'] == anno and item['Nil'] == quartiere:
                newkey = item['regione'] + ' M'
                if newkey not in newrecord:
                    newrecord[newkey] = item['Maschi']
                    newrecord[item['regione'] + ' F'] = item['Femmine']
                    newrecord[item['regione'] + ' Tot'] = item['Maschi'] + item['Femmine']
                else:
                    newrecord[item['regione'] + ' M'] += item['Maschi']
                    newrecord[item['regione'] + ' F'] += item['Femmine']
                    newrecord[item['regione'] + ' Tot'] += (item['Maschi'] + item['Femmine'])      
        compactData.append(newrecord)

print(compactData[:4])
print(len(compactData[0]))

Salvo anche questo:

In [ ]:
import json
dirPath2 = '/Users/albertocottica/Documents/GitHub/-milano-melting-pot/dati/'
with open (dirPath2 + 'map1Data_alt.json', 'w') as outfile:
    json.dump(compactData2, outfile)
print('Done')